# Channels' tags network

In [3]:
import os
import sys
import nltk
import math
import json
import shlex
import itertools
import networkx as nx
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from string import punctuation

from pandas import Series
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer

In [37]:
# loading data
data_folder_channels='/media/aruiz/data/channels_clean_data/'
fileName2=os.path.join(data_folder_channels,'all_channels_cat_clean.csv')
# fileName2=os.path.join(data_folder_channels,'gta_data_clean.csv')
df=pd.read_csv(fileName2)
df=df.sort_values(by=['views'],ascending=True)
df=df.fillna("N/A")
df=df[df['views']>42267]
df=df.reset_index(drop=True)
lemmatizer = WordNetLemmatizer()
stopWords = set(stopwords.words('english'))
searchKeys=["minecraft","roblox","call", "of", "duty","overwatch", "grand","theft","auto","league","legend",
            "Happy","wheel","five", "night", "at", "freddy's","agar.io", "pokemon"]
# searchKeys=["grand","theft","auto"]
# top_words= ["game", "gaming", "video", "gameplay", "funny", "play", "let","cod", "mod", "tutorial"]
# quartile3=246352
quartile3=42267
#punctuation list to exclude from keywords
exclude = set(punctuation)

/home/aruiz/.virtualenvs/LetsPlayData/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Write keywords per channel in a file

In [6]:
#Method to delete numbers and punctuation signs
def remove_dig_punct(list1):
    new=[]
    for n in range (0, len(list1)):
        #removing digits
        res=''.join([i for i in list1[n] if not i.isdigit()])
        #removing punctuation
        s = ''.join(ch for ch in res if ch not in exclude)
        if (s !="") and (s not in new):
            if (len(s)>1):
                new.append(s)
    return (new)

In [27]:
with open("channel_id_tags.json", 'w') as f:
    for n in range(0,len(df)):
        if (df.keywords[n]!="N/A") and (df.views[n]>quartile3):
            d1={}
            channel_keys=[]

            try:
                l = shlex.split(df.keywords[n])

            except:
                l = df.keywords[n].split()

            for kw in l:
                kw_id = kw.lower().replace("'", "").replace("´", "").replace("’", "").replace("`", "")
                single_keys=nltk.word_tokenize(kw_id)
                for key in single_keys:
                    key=lemmatizer.lemmatize(key)
                    if (key not in stopWords)and(len(key)>1):
                        if (key not in channel_keys) and (key not in searchKeys):
                            channel_keys.append(key)

    #         clean all digits and punctuation from keywords
            channel_keys_clean=remove_dig_punct(channel_keys)

    #         df.loc[n,"num_tags"]=len(channel_keys_clean)
            for key in channel_keys_clean: d1[key]=True
            d1["id"]=df.id[n]
#             dictionaries.append(d1)
            f.write(json.dumps(d1))
            f.write('\n')

### Construct graph network with weighted edges (w = number of shared tags)

In [4]:
# create nodes with weight and title
G=nx.Graph()
with open("channel_id_tags.json", 'r') as f:
    for line in f:
        comp1=json.loads(line)
        channel=df.loc[df["id"]==comp1["id"],["title", "views", "game_q"]]
        if len(str(channel.game_q.values[0]))==1:
            cat=int(channel.game_q.values[0])
        else:
            cat=10
        G.add_node(comp1["id"], Label=channel.title.values[0], weight=int(channel.views.values[0]),category=cat)       

KeyboardInterrupt: 

In [ ]:
def return_line_dict(n, filename):
    
    with open(filename, 'r') as f:
        comp1={}
        for i,line in enumerate(f):
            if i==n:
                comp1=json.loads(line)
                break
    f.close()
    return comp1

In [5]:
for n in range (0,22415):
    comp1= return_line_dict(n,"channel_id_tags.json")
    with open("channel_id_tags.json", 'r') as f:
        for j,line in enumerate(f):
            if j>=n+1:
                comp2=json.loads(line)
                shared_items = set(comp1.items()) & set(comp2.items())
                shared_len=len(shared_items)
                if (shared_len>=5):
                    G.add_edge(comp1["id"],comp2["id"], weight=int(shared_len), shared=shared_items)
        f.close()

## Merge nodes 

In [149]:
def merge_nodes(new_node, node_list, gf):
    H =gf.copy()
    gf.add_node(new_node, type='keyword') # Add the 'merged' node   

    for n1,n2,data in H.edges(data=True):
        # For all edges related to one of the nodes to merge,
        # make an edge going to or coming from the `new gene`.
        if n1 in node_list:
            g.add_edge(new_node,n2)
        elif n2 in node_list:
            g.add_edge(n1,new_node)

    for n in node_list: # remove the merged nodes
        gf.remove_node(n)

### Remove isolates and keep strong edges

In [7]:
H=G.copy()
a=list(nx.isolates(G))
H.remove_nodes_from(a)

In [8]:
#change edge relations data to dictionary
for u,v,d in H.edges(data=True):
    d['shared']=dict(d['shared'])

In [43]:
J=H.copy()
cum_sum=0

for u,v,d in H.edges(data=True):
    if d['weight']<8:
        J.remove_edge(u,v)
#         cum_sum+=1
#         print(u,"",v, d['weight'])

In [45]:
a=list(nx.isolates(J))
J.remove_nodes_from(a)

In [13]:
test=H.copy()
print(len(test.nodes()))
print(len(test.edges()))
print(len(list(nx.isolates(test))))

7724
317813
0


In [10]:
distribution=[]
for u,v,d in H.edges(data=True):
    distribution.append(d['weight'])

In [9]:
nx.write_gexf(H,"similarity_channelstags5_64data.gexf")

In [151]:
H=[]
K=[]
L=[]

In [11]:
counts_w=Series(distribution)
rep=counts_w.value_counts()

In [12]:
rep

5     182865
6      74900
7      32475
8      14279
9       6585
10      3285
11      1585
12       767
13       418
14       232
15       140
16        85
17        41
18        38
19        28
20        18
21        12
25        12
22         9
27         8
23         6
29         3
26         3
35         2
24         2
28         2
30         2
37         2
32         2
33         2
34         2
40         1
31         1
64         1
dtype: int64

In [1]:
def assign_keys(key_string):
    channel_keys_clean=''
    if (key_string!="N/A"):
        channel_keys=[]

        try:
            l = shlex.split(key_string)

        except:
            l = key_string.split()

        for kw in l:
            kw_id = kw.lower().replace("'", "").replace("´", "").replace("’", "").replace("`", "")
            single_keys=nltk.word_tokenize(kw_id)
            for key in single_keys:
                key=lemmatizer.lemmatize(key)
                if (key not in stopWords)and(len(key)>1):
                    if (key not in channel_keys) and (key not in searchKeys):
                        channel_keys.append(key)

#         clean all digits and punctuation from keywords
        channel_keys_clean=remove_dig_punct(channel_keys)
#         channel_keys_clean=' '.join (channel_keys_clean)
#         df.loc[n,"num_tags"]=len(channel_keys_clean)
    return channel_keys_clean

In [40]:
df['temp']=df.keywords.apply(lambda x: assign_keys(x))

In [39]:
df=df[df['temp']!='']
df=df.reset_index(drop=True)

KeyError: 'temp'

In [38]:
len(df)

34571

In [31]:
#Nodes
g=nx.Graph()
for i in range (0, len(df)):
    g.add_node(df.id[i], Label=df.title[i],weight=int(df.views[i]), category=df.game_q[i])

In [33]:
%%time
#edges
for i in range (0, len(df)-1):
    for j in range (i+1,len(df)):
        shared_items=set(df.temp[i]).intersection(set(df.temp[j]))
        shared_len=len(shared_items)
        item_string=' '.join(list(shared_items))
        if shared_len>8 and shared_len<64:
            g.add_edge(df.id[i],df.id[j], weight=int(shared_len), shared=item_string)

KeyboardInterrupt: 

In [32]:
print(len(g.nodes()))
print(len(g.edges()))

22415
0


In [ ]:
h=g.copy()
a=nx.isolates (g)
h.remove_nodes_from(a)

In [ ]:
print(len(h.nodes()))
print(len(h.edges()))

In [36]:
df.head()

,id,title,description,year_created,month_created,views,subscribers,videos,comments,keywords,game_q,temp
0,UCgRSExu_Dr7mgsqSHsu1iIA,SeRaGameplay,N/A,2012,2,42280,698,102,12,SeRaGameplay,9,[seragameplay]
1,UC4wlTpHcCuIJlGY0K1dMpeg,Bouke,My youtube account:\r\nI am well gamers name: ...,2007,8,42283,92,33,5,Flessenkop Mario Worker Constructor call of du...,4,"[flessenkop, mario, worker, constructor, machi..."
2,UCyuYSx_n4-9JQANMuq5KtkA,82ndABKhan,_____________________________\r\n\r\nIf you pl...,2007,2,42304,34,6,5,cod call of duty cod2 cod4 codz 82nd 82ndab cl...,4,"[cod, codz, nd, ndab, clan, pwn, newb, nub, vi..."
3,UCl0svB7y3vrcvd79IzlsbvA,skitzmaker,Hi guys welcome to my channel! Make sure to SU...,2007,6,42325,219,141,0,Flappybirdys,9,[flappybirdys]
4,UCZRFLy_KpcM2_7C_6vzNwBw,Salt,Zombies live streams with different challenges...,2009,5,42355,1113,26,16,Thungergun ZombieRecords ZombieLeaderboards PC...,4,"[thungergun, zombierecords, zombieleaderboards..."
